In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/home/gregory_maruss/VESUVIUS_Challenge/jupyter notebooks
Current path:/home/gregory_maruss/VESUVIUS_Challenge


In [2]:
import torch
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Tile_Dataset import Vesuvius_Tile_Datamodule
import matplotlib.patches as patches
from lit_models.UNET_TILE import UNET_TILE_lit
from monai.visualize import matshow3d
import einops
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

2023-05-07 05:06:57,213 - Created a temporary directory at /tmp/tmp3y9193_d
2023-05-07 05:06:57,215 - Writing /tmp/tmp3y9193_d/_remote_module_non_scriptable.py


In [3]:
Z_DIM = 48
PATCH_SIZE =256

class CFG:
    
    train_fragment_id=[2,3]
    val_fragment_id=[1]
    batch_size = 16
    patch_size = PATCH_SIZE
    z_dim = Z_DIM
    stride = patch_size // 2
    #comp_dataset_path = COMPETITION_DATA_DIR
    num_workers = 8
    on_gpu = True

In [4]:
dataset = Vesuvius_Tile_Datamodule(cfg=CFG)

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

In [5]:
lit_model = UNET_TILE_lit(
        use_wandb = True,
        z_dim = Z_DIM,
        patch_size = (PATCH_SIZE,PATCH_SIZE),
        sw_batch_size=8 ,
        eta_min = 5e-7,
        t_max = 60,
        max_epochs = 80,
        weight_decay =  0.0001,
        learning_rate = 0.0001,
        gamma = 0.85,)

FROM_CHECKPOINT = False
if FROM_CHECKPOINT:
    lit_model = lit_model.load_from_checkpoint('logs/AUG_MonaiFlexUnet_5layers768_BCE1_Tile_16_256patch/last.ckpt', 
                                             #patch_size = (512,512),
                                             learning_rate = 0.00001,
                                              weight_decay =  .00001,
                                              eta_min = 1e-7,
                                               t_max = 60,
                                             sw_batch_size = 8)

wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [6]:


checkpoint_callback = ModelCheckpoint(
    save_top_k=7,
    monitor="FBETA",
    mode="max",
    dirpath="logs/48z_BCE04+05Tver_MUnetB4_64to768/",
    filename="48z_BCE04+05Tver_MUnetB4_64to768{epoch:02d}{val_loss:.2f}{FBETA:.2f}{fbeta_4:.2f}{fbeta_6:.2f}{precision:.2f}{recall:.2f}",
    save_last =True,
)


trainer = pl.Trainer(
        accelerator='gpu',
        #benchmark=True,
        max_epochs=90,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/48z_BCE04+05Tver_MUnetB4_64to768/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/48z_BCE04+05Tver_MUnetB4_64to768/',
        #overfit_batches=1,
        precision=16,
        accumulate_grad_batches=12, 
        callbacks=[checkpoint_callback],
        resume_from_checkpoint ='logs/48z_BCE04+05Tver_MUnetB4_64to768/last.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/Full_16_768to1024_batch/FocalDice_768epoch=168FBETA=0.54recall=0.45precision=0.57.ckpt'
           )
# resume_from_checkpoint = 
#ckpt_path='logs/unet_smp-epoch=102-val_loss=0.00.ckpt'
# ckpt_path='logs/Eff_monai_32z/lightning_logs/version_0/checkpoints/epoch=19-step=40.ckp'

2023-05-07 05:10:42,187 - Using 16bit None Automatic Mixed Precision (AMP)
2023-05-07 05:10:42,245 - GPU available: True (cuda), used: True
2023-05-07 05:10:42,246 - TPU available: False, using: 0 TPU cores
2023-05-07 05:10:42,247 - IPU available: False, using: 0 IPUs
2023-05-07 05:10:42,248 - HPU available: False, using: 0 HPUs


/home/gregory_maruss/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(


2023-05-07 05:10:42,479 - Restoring states from the checkpoint path at logs/48z_BCE04+05Tver_MUnetB4_64to768/last.ckpt


/home/gregory_maruss/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1906: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  rank_zero_deprecation(
/home/gregory_maruss/anaconda3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/gregory_maruss/VESUVIUS_Challenge/logs/48z_BCE04+05Tver_MUnetB4_64to768 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


2023-05-07 05:10:44,334 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Adjusting learning rate of group 0 to 1.0000e-04.
2023-05-07 05:10:44,352 - 
  | Name                  | Type                  | Params
----------------------------------------------------------------
0 | metrics               | ModuleDict            | 0     
1 | model                 | FlexibleUNet          | 35.7 M
2 | loss_dice             | DiceLoss              | 0     
3 | loss_tversky          | TverskyLoss           | 0     
4 | loss_focal            | FocalLoss             | 0     
5 | loss_bce              | SoftBCEWithLogitsLoss | 0     
6 | loss_monai_focal_dice | DiceFocalLoss         | 0     
----------------------------------------------------------------
35.7 M    Trainable params
0         Non-trainable params
35.7 M    Total params
71.401    Total estimated model params size (MB)
2023-05-07 05:10:44,861 - Restored all states from the checkpoint file at logs/48z_BCE04+05Tver_MUnetB4_64to768/last.ckpt


...

Adjusting learning rate of group 0 to 1.8941e-05.


Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 1.6961e-05.


Validation: 0it [00:00, ?it/s]

/home/gregory_maruss/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
